In [99]:
from qiskit import QuantumCircuit
from math import pi
from mitiq import Observable, PauliString
from mitiq.interface.mitiq_qiskit import qiskit_utils
from qiskit import execute, Aer
import qiskit.providers.aer.noise as noise

def get_circuit():
    qc = QuantumCircuit(2)

    # CDR works better if the circuit is not too short. So we increase its depth.
    for i in range(5): 
        qc.h(0) # Clifford
        qc.h(1) # Clifford
        qc.rz(1.75, 0)
        qc.rz(2.31, 1)
        qc.cx(0,1) # Clifford
        qc.rz(-1.17, 1)
        qc.rz(3.23, 0)
        qc.rx(pi/2, 0) # Clifford
        qc.rx(pi/2, 1) # Clifford

    # We need to measure the qubits
    #qc.measure_all()
    return qc

In [100]:
obsv = Observable(PauliString("ZZ"), PauliString("X", coeff=-1.75))
print(obsv)

Z(q(0))*Z(q(1)) + (-1.75+0j)*X(q(0))


In [101]:
# 2. Define an executor
from mitiq.interface.mitiq_qiskit import qiskit_utils
from qiskit import QuantumCircuit, execute, Aer
import qiskit.providers.aer.noise as noise
# Error probabilities
prob_1 = 0.005  # 1-qubit gate
prob_2 = 0.01   # 2-qubit gate

# Depolarizing quantum errors
error_1 = noise.depolarizing_error(prob_1, 1)
error_2 = noise.depolarizing_error(prob_2, 2)

# Add errors to noise model
noise_model = noise.NoiseModel()
noise_model.add_all_qubit_quantum_error(error_1, ['u1', 'u2', 'u3'])
noise_model.add_all_qubit_quantum_error(error_2, ['cx'])

def sim_noise(qc):
    return qiskit_utils.execute_with_shots_and_noise(qc, obsv, noise_model, 4096)

In [102]:
def sim(qc):
    return qiskit_utils.execute_with_shots(qc, obsv, 4096)
def sim2(qc):
    return qiskit_utils.execute(qc, obsv.matrix())

In [103]:
#obs2 = Observable(None)
ideal_measurement = obs.expectation(get_circuit, sim2).real
print("ideal_measurement = ",ideal_measurement)

ValueError: Executor and observable are not compatible. Executors
                returning expectation values as float must be used with
                observable=None

In [ ]:
unmitigated_measurement = obs.expectation(get_circuit(), sim_noise).real
print("unmitigated_measurement = ", unmitigated_measurement)

ValueError: Executor and observable are not compatible. Executors
                returning expectation values as float must be used with
                observable=None

In [84]:
from mitiq import cdr

mitigated_measurement = cdr.execute_with_cdr(
    get_circuit(),
    sim_noise,
    observable=obs,
    simulator=sim,
    seed=0,
).real
print("mitigated_measurement = ", mitigated_measurement)

ValueError: Executor and observable are not compatible. Executors
                returning expectation values as float must be used with
                observable=None

In [ ]:
error_unmitigated = abs(unmitigated_measurement-ideal_measurement)
error_mitigated = abs(mitigated_measurement-ideal_measurement)

print("Error (unmitigated):", error_unmitigated)
print("Error (mitigated with CDR):", error_mitigated)

print("Relative error (unmitigated):", (error_unmitigated/ideal_measurement))
print("Relative error (mitigated with CDR):", error_mitigated/ideal_measurement)

print(f"Error reduction with CDR: {(error_unmitigated-error_mitigated)/error_unmitigated :.1%}.")

NameError: name 'unmitigated_measurement' is not defined

In [110]:
import numpy as np
import matplotlib.pyplot as plt
gammas = np.linspace(0, 2 * np.pi, 50)

In [111]:
plt.figure(figsize=(8, 6))
plt.plot(gammas, noiseless_expectations, color="g", linewidth=3, label="Noiseless")
plt.title("Energy landscape", fontsize=16)
plt.xlabel(r"Ansatz angle $\gamma$", fontsize=16)
plt.ylabel(r"$\langle H \rangle(\gamma)$", fontsize=16)
plt.legend(fontsize=14)
plt.ylim(-1, 1);
plt.show()

NameError: name 'noiseless_expectations' is not defined

<Figure size 800x600 with 0 Axes>